In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.datasets import load_files
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Problem Statement

Estimate
1.  Type of restrants,dishes,cuisines,cost for dinning preffered
2.  Location famous for online orders and book_table(what rest_type)
3.  Places with common menus and unique dishes.
4.  Use the voting,rate to estimate the best restrant in bangalores
5.  Sentimental analysis (review list)
6.  Estimating the price with the help of model prediction

In [ ]:
zomato = pd.read_csv("/kaggle/input/zomato-bangalore-restaurants/zomato.csv")
zomato.head()

In [ ]:
zomato.info()

In [ ]:
# Percentage of Nul Values
pd.DataFrame(round(zomato.isnull().sum()/zomato.shape[0] * 100,3), columns = ["Nan"])  

### Dropping the columns 

- Ideas to fill the null values
- Lets drop dish_liked as it contains more than 50% null values
- Drop the phone number as we cannot manipulate them
- approx_cost-- We can try to cluster it according to its neighbors price or using the mean values
- Rate - We try to fill after analysing the reveiws_list

In [ ]:
zomato=zomato.drop(['url','dish_liked','phone'],axis=1)

### Altering the column Names as required

In [ ]:
zomato.columns

In [ ]:
zomato= zomato.rename(columns={"approx_cost(for two people)" : "Cost_of_two",
                                "reviews_list": "reviews",
                                 "menu_item"  : "menu",
                                 "listed_in(type)": "type",
                                 "listed_in(city)": "city"})

### Changes with respect to the rate and the cost

In [ ]:
zomato.head()

In [ ]:
zomato['rate']=zomato['rate'].str.split('/').str[0]

### Filling the NEW shops and null values with the rating based on the reviews

In [ ]:
sd= zomato.copy(deep=True)
sd= zomato[zomato['rate']=='NEW']
fd= zomato[zomato['rate'].isna()]
sfd= pd.concat([sd,fd])
sfd= sfd[sfd['reviews']!=" "] ## Making sure that the review is not empty

In [ ]:
import nltk
nltk.download("stopwords")

stopwords= nltk.corpus.stopwords.words('english')
stopwords.remove('not')
stemmer= nltk.stem.PorterStemmer()
import re

def clean_data(doc):
    doc= doc.lower()
    doc= re.sub(r"\W"," ",doc)
    doc= re.sub(r"\d"," ",doc)
    doc= re.sub(r"\s+"," ",doc)
    doc=re.sub("[^a-z\s]","",doc)
    words= doc.split(" ")
    doc= re.sub(r" [@#$%\&\*\(\)\<\>\?\'\":;\]\[-] ", " ",doc)
    word_imp= [ stemmer.stem(word) for word in words if word not in stopwords]
    doc_cleaned=" ".join(word_imp)
    return doc_cleaned
sid= SentimentIntensityAnalyzer()
sfd["reviews"]=sfd["reviews"].apply(clean_data)

In [ ]:
sfd['scores'] = sfd['reviews'].apply(lambda reviews: sid.polarity_scores(reviews))

sfd['compound']  = sfd['scores'].apply(lambda score_dict: score_dict['compound'])

sfd['neu']  = sfd['scores'].apply(lambda score_dict: score_dict['neu'])

sfd['comp_score'] = sfd['compound'].apply(lambda c: 'pos' if c >=0.2 else 'neg')

sfd.head()

In [ ]:
sfd["comp_score"].value_counts()

- Assumption NEW starts lets be good as they are trying new
- Compound_Score with score greater than 0.2 will be given positive with a rating as 4 
- Lesser than 0.2 is given negative with a rating as 3

In [ ]:
def comp(comp_score):
    if comp_score =='pos':
        return 4
    else:
        return 3
sfd['new_rate']=sfd['comp_score'].apply(comp)

In [ ]:
nsd= sfd.drop(['scores','compound','neu','rate','comp_score'],1)
nsd=nsd.rename(columns={"new_rate":"rate"})
nzomato= zomato.loc[(zomato['rate']!='NEW')]
nzomato =nzomato.loc[zomato['rate'].notna()]
frame=[nzomato,nsd]
final=pd.concat(frame)

In [ ]:
final.head()

### Fixing Cost_of_Two

In [ ]:
final= final[final['rate']!='-']
final['Cost_of_two']=final['Cost_of_two'].replace(",","",regex=True).astype(float)

In [ ]:
final['Cost_of_two'].plot(kind='hist',figsize=(8,8))

In [ ]:
### Adjustments on the data types
final['rate']=final['rate'].astype("float")
# round(final['Cost_of_two'].mean()) ---- > 586
final['Cost_of_two']=final['Cost_of_two'].fillna('586')
final['rest_type']= final['rest_type'].fillna('Quick Bites')  # Filled with mode(maximum occurance)

### Cuisines

In [ ]:
menu= final[final['cuisines'].isna()]
# There are around 10 restrants without the cuisines type we can fill after analysing the menu enterily

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
WC= menu[menu['menu'].map(lambda d: len(d)) > 5]
# Start with one menu:
# Create and generate a word cloud image:
for i in range(0,4):
    text= WC.menu.iloc[i]
    wordcloud = WordCloud().generate(text)

# Display the generated image:
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(WC['name'].iloc[i])
    plt.show()

In [ ]:
final.loc[(pd.isnull(final.cuisines)), 'cuisines'] = "Briyani"
final.loc[(final.name=="Lassi Spot"),"cuisines"]= "Desserts, Ice Cream "

In [ ]:
final.isnull().sum()

### Data is clean for futher processing

### Univariate Analysis 

In [ ]:
# Top 20 shops in the zomato list
final['name'].value_counts()[:20].plot(kind='bar')

In [ ]:
# Bangalore a IT hub everone needs a quick bites (reallly sad)
final['rest_type'].value_counts()[:20].plot(kind='bar')

In [ ]:
plt.figure(figsize=(14,4))
plt.grid()
sns.distplot(final['Cost_of_two'])
#final['Cost_of_two'].value_counts().sort_values(ascending=False).plot(kind='bar')
plt.show()

Most of them are spending on zomato food within the price range of 200-1000

### Briyani Shops


In [ ]:
print(final[final['name'].str.lower().str.contains("briyani")].shape)
Briyani= final[final['name'].str.lower().str.contains("briyani")]
Briyani.shape

In [ ]:
Briyani.groupby('name')['location'].count().plot(figsize=(8,8),kind='pie')

In [ ]:
Briyani.groupby('location')['location'].count().plot(figsize=(8,8),kind='pie')

In [ ]:
Briyani.groupby('location')['Cost_of_two'].mean().plot(figsize=(12,8),kind='line')
# Briyani is costly in IndiraNagar and lesser in JP Nagar,Peenya

In [ ]:
Briyani.groupby('online_order')['location'].count()

Inference:

- There is around 35 briyani centers and top 5 unique briyani shops spreaded across
- Ambur star briyani is one with the more count compared to other briyani centers
- While karnatka briyani point is unique to a particular location

- BTM has a highest briyani foods(Briyani lovers it the place to chill)
- Price range of briyani varies from 200-500
- Most of them with online orders

### All the missing datas are imputed and ready for Prediction of rate

In [ ]:
# Getting dummies is not going to work here as we have columns with address,name,menu
# Using factorize with reference to one of the kaggle kernels

def Encode(zomato):
    for column in zomato.columns[~zomato.columns.isin(['rate', 'Cost_of_two', 'votes'])]:
        zomato[column] = zomato[column].factorize()[0]
    return zomato

zomato_en = Encode(final.copy())

In [ ]:
zomato_en

In [ ]:
zomato_en['Cost_of_two']=zomato_en['Cost_of_two'].astype(int)

In [ ]:
X= zomato_en.drop('Cost_of_two',axis=1)
y= zomato_en['Cost_of_two']

### Using backward (wrapper method) to select the feature

In [ ]:
import scipy.stats as st
import statsmodels.api         as     sm

In [ ]:
#Adding constant column of ones, mandatory for sm.OLS model
X_1 = sm.add_constant(X)
#Fitting sm.OLS model
model = sm.OLS(y,X_1).fit()
model.pvalues
#model.summary()

In [ ]:
#Backward Elimination
cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

In [ ]:
X= X[selected_features_BE]

In [ ]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,accuracy_score
from sklearn.model_selection import train_test_split
import xgboost

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

In [ ]:
RFR=RandomForestRegressor()
RFR.fit(X_train,y_train)

GB= GradientBoostingRegressor()
GB.fit(X_train,y_train)

Ada= AdaBoostRegressor()
Ada.fit(X_train,y_train)

XG= xgboost.XGBRegressor()
XG.fit(X_train,y_train)

In [ ]:
RFR_train_pred=RFR.predict(X_train)
GB_train_predict=GB.predict(X_train)
Ada_train_predict=Ada.predict(X_train)
XG_train_predict=XG.predict(X_train)

In [ ]:
RFR_test_pred=RFR.predict(X_test)
GB_test_predict=GB.predict(X_test)
Ada_test_predict=Ada.predict(X_test)
XG_test_predict=XG.predict(X_test)

In [ ]:
## r2_score

r2_score_train=[r2_score(y_train,RFR_train_pred),r2_score(y_train,GB_train_predict),r2_score(y_train,Ada_train_predict),r2_score(y_train,XG_train_predict)]
r2_score_test =[r2_score(y_test,RFR_test_pred),r2_score(y_test,GB_test_predict),r2_score(y_test,Ada_test_predict),r2_score(y_test,XG_test_predict)]
a=pd.DataFrame({'r2_score_train':r2_score_train,'r2_score_test':r2_score_test})
plt.figure(figsize=[10,5]) 
a.plot(kind='bar')
plt.xticks(np.arange(4),['RFR','GB','ada','XG'])
plt.show()

## Overall RandomForest and XGboost are performing best in the prediction

### Beginners Code open for suggestions.. Thanks